In [ ]:
from map_util import *
from play import GamePlayer
from time import sleep

In [ ]:
def fixed():
    with GamePlayer() as player:
        player.start(6)
        #sequence = 'WNWNWNWNNEEESSSSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEEENNEEEEEeeeeESSSSSSeEE'
        sequence = 'WNWNWNWNNEEEEEEESWSWSWSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEENENEEeeeeeeESSSSSSEEeEE'
        #sequence = 'WNWNWNWNNEEESSSSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEEENNEEESNSNSNS'

        for c in sequence:
            if c == 'N': player.move('N')
            if c == 'E': player.move('E')
            if c == 'S': player.move('S')
            if c == 'W': player.move('W')
            if c == 'n': player.use('N')
            if c == 'e': player.use('E')
            if c == 's': player.use('S')
            if c == 'w': player.use('W')
            
            if player.finished:
                print('FINISHED')
                break
            #sleep(0.1)

#fixed()

In [ ]:
def valid_pos(pos):
    return pos[0] >= 0 and pos[1] >= 0

def can_act1():
    global player
    return valid_pos(player.player1_pos) and player.action1 is None

def can_act2():
    global player
    return valid_pos(player.player2_pos) and player.action2 is None    

def move_to_1(pos):
    global player
    dir = player.get_direction_from_to(player.player1_pos, pos, player.player1_paths, player.player1_paths_nokeys)
    player.queue_move1(dir)

def move_to_2(pos):
    global player
    dir = player.get_direction_from_to(player.player2_pos, pos, player.player2_paths, player.player2_paths_nokeys)
    player.queue_move2(dir)

def use_1(pos):
    global player
    if player.player1_pos[0] < pos[0]: player.queue_use1('S')
    if player.player1_pos[0] > pos[0]: player.queue_use1('N')
    if player.player1_pos[1] < pos[1]: player.queue_use1('E')
    if player.player1_pos[1] > pos[1]: player.queue_use1('W')

def use_2(pos):
    global player
    if player.player2_pos[0] < pos[0]: player.queue_use2('S')
    if player.player2_pos[0] > pos[0]: player.queue_use2('N')
    if player.player2_pos[1] < pos[1]: player.queue_use2('E')
    if player.player2_pos[1] > pos[1]: player.queue_use2('W')


In [ ]:
def get_front_door(door_pos, distances):
    global player
    top = (door_pos[0]-1, door_pos[1])
    if top in distances: return top
    bottom = (door_pos[0]+1, door_pos[1])
    if bottom in distances: return bottom
    left = (door_pos[0], door_pos[1]-1)
    if left in distances: return left
    right = (door_pos[0], door_pos[1]+1)
    if right in distances: return right
    return None
    
def pickup_key_or_open_door(key, door, item):
    global player
    doors = np.argwhere(player.map == door)
    if np.any(doors):
        door_pos = tuple(doors[0])
        if can_act1() and player.player1_inventory == item:
            target_pos = get_front_door(door_pos, player.player1_distances_nokeys)
            if player.player1_pos != target_pos:
                print('move_door1')
                move_to_1(target_pos)
            else:
                print('use_door1')
                use_1(door_pos)
        elif can_act2() and player.player2_inventory == item:
            target_pos = get_front_door(door_pos, player.player2_distances_nokeys)
            if player.player2_pos != target_pos:
                print('move_door2')
                move_to_2(target_pos)
            else:
                print('use_door2')
                use_2(door_pos)
        else:
            keys = np.argwhere(player.map == key)
            if np.any(keys):
                key_pos = tuple(keys[0])
                if can_act1() and player.player1_inventory == 0:
                    print('move_key1')
                    move_to_1(key_pos)
                elif can_act2() and player.player2_inventory == 0:
                    print('move_key2')
                    move_to_2(key_pos)

In [ ]:
with GamePlayer() as player:
    player.start(9)

    while not player.finished:
        
        # Move to exit if possible
        exits = np.argwhere(player.map == EXIT)
        if np.any(exits):
            exit_pos = tuple(exits[0])
            if can_act1():
                print('exit1')
                move_to_1(exit_pos)
            if can_act2():
                print('exit2')
                move_to_2(exit_pos)

        # Pickup health
        healths = np.argwhere(player.map == HEALTH)
        if np.any(healths):
            health_pos = tuple(healths[0])
            if can_act1() and player.player1_health < 5:
                print('health1')
                move_to_1(health_pos)
            elif can_act2() and player.player2_health < 5:
                print('health2')
                move_to_2(health_pos)

        # Pickup sword
        swords = np.argwhere(player.map == SWORD)
        if np.any(swords):
            sword_pos = tuple(swords[0])
            if can_act1() and not player.player1_has_sword:
                print('sword1')
                move_to_1(sword_pos)
            elif can_act2() and not player.player2_has_sword:
                print('sword2')
                move_to_2(sword_pos)
                
        # Pickup keys
        pickup_key_or_open_door(KEY_RED, DOOR_RED, INVENTORY_KEY_RED)
        pickup_key_or_open_door(KEY_GREEN, DOOR_GREEN, INVENTORY_KEY_GREEN)
        pickup_key_or_open_door(KEY_BLUE, DOOR_BLUE, INVENTORY_KEY_BLUE)

        # Explore
        if can_act1():
            print('explore1')
            dir1 = player.get_direction_towards_closest_unknown(player.player1_pos, player.player1_distances_nokeys, player.player1_paths_nokeys)
            player.queue_move1(dir1)
        if can_act2():
            print('explore2')
            dir2 = player.get_direction_towards_closest_unknown(player.player2_pos, player.player2_distances_nokeys, player.player2_paths_nokeys)
            player.queue_move2(dir2)
        
        player.act()
        
        
        #player.step()
        #sleep(0.2)